In [1]:
# cd 先进入当前路径
# tensorboard --logdir logs

In [1]:
import tensorflow as tf
from tensorflow import keras


In [5]:
# 加载数据
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 数据预处理
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    y = tf.cast(y, dtype=tf.int32)
    return x, y
    

In [7]:
# 划分训练接和测试集
batch_size = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).batch(batch_size)

In [11]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10)
])

# [None, 28 * 28] none 代表样本数量不确定 可以是任意个样本, 样本必须是28 * 28的
model.build(input_shape=[None, 28 * 28])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             multiple                  401920    
_________________________________________________________________
dense_13 (Dense)             multiple                  131328    
_________________________________________________________________
dense_14 (Dense)             multiple                  32896     
_________________________________________________________________
dense_15 (Dense)             multiple                  8256      
_________________________________________________________________
dense_16 (Dense)             multiple                  2080      
_________________________________________________________________
dense_17 (Dense)             multiple                  330       
Total params: 576,810
Trainable params: 576,810
Non-trainable params: 0
________________________________________________

In [2]:
# 优化器
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# 评估指标 
# update_state 更新状态
# reset_states 清除缓存 否则会记录上一次的保存的值
accuracy_metrics = keras.metrics.Accuracy()
mean_metrics = keras.metrics.Mean()

epochs = 10
for epoch in range(epochs):
    for step, (x, y) in enumerate(db_train):
        x = tf.reshape(x, [-1, 28 * 28])
        y = tf.one_hot(y, depth=10, dtype=tf.int32)
        with tf.GradientTape() as tap:
            y_pred = model(x)

            loss = tf.reduce_mean(keras.losses.categorical_crossentropy(y, y_pred, from_logits=True))
            mean_metrics.update_state(loss)
            
        grads = tap.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if step % 100 == 0:
            print(step, 'loss:', mean_metrics.result().numpy())
            # 清除缓存 否则会进行累积
            mean_metrics.reset_states()
    
    # 测试集上测试泛化能力
    total_correct, total_num = 0, 0
    accuracy_metrics.reset_states()

    for x, y in db_test:
        x = tf.reshape(x, [-1, 28 * 28])
        
        y_pred = model(x)
        # 将预测出来的标签值 转化为概率
        y_prob = tf.nn.softmax(y_pred)
        y_pred = tf.argmax(y_prob, axis=1)
        y_pred = tf.cast(y_pred, dtype=tf.int32)
        correct = tf.equal(y, y_pred)
        correct = tf.cast(correct, dtype=tf.int32)
        
        total_correct += tf.reduce_sum(correct)
        total_num += x.shape[0]
        
        accuracy_metrics.update_state(y, y_pred)
    
    # 怪不得我刚才得到的正确率不一样 原来是因为我没有清除上一次迭代中accuracy_metrics
    print('【正确率 aucc = %f】' % (total_correct / total_num))
    print('【正确率 accuracy_metrics = %f】' % (accuracy_metrics.result().numpy()))    
    

NameError: name 'keras' is not defined

(128, 28, 28)
